# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


City_ID        City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0      wismar  53.9000   11.4667     31.24        89         100   
1        1   inhambane -23.8650   35.3833     73.02        67           0   
2        2  borogontsy  62.6683  131.1683    -33.83       100         100   
3        3   adamstown -25.0660 -130.1015     71.80        82           8   
4        4  constantia  44.1833   28.6500     56.08        79          78   

   Wind Speed Country        Date  
0        1.01      DE  1701541765  
1       11.01      MZ  1701541767  
2        2.71      RU  1701541768  
3       14.50      PN  1701541550  
4       13.69      RO  1701541770

In [3]:
import requests
from requests.structures import CaseInsensitiveDict

url = "https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=rect%3A10.716463143326969%2C48.755151258420966%2C10.835314015356737%2C48.680903341613316&limit=20&apiKey=3017c7a140da4c4bad32be4fdae1478a"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)

200


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size = "Humidity",
    scale = 1,
)


# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] <= 78) & (city_data_df['Max Temp'] >= 60)
                                 & (city_data_df['Humidity'] <= 74) & (city_data_df['Humidity'] <= 60)]

# Drop any rows with null values
# YOUR CODE HERE
ideal_city_df = ideal_city_df.dropna(how="any")

# Display sample data
# YOUR CODE HERE
ideal_city_df.head()

City_ID           City      Lat      Lng  Max Temp  Humidity  Cloudiness  \
43       43         bechar  31.6167  -2.2167     62.92        31          20   
45       45         rawson -43.3002 -65.1023     63.50        18         100   
65       65       tolanaro -25.0319  46.9987     73.47        60          75   
88       88  sierra grande -41.6060 -65.3557     64.80         9         100   
98       98      kalinagar  28.6167  80.1000     61.86        57           0   

    Wind Speed Country        Date  
43        9.22      DZ  1701541813  
45       21.32      AR  1701541818  
65       13.80      MG  1701541844  
88       23.98      AR  1701541870  
98        6.62      IN  1701541887

### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.head()

City Country      Lat      Lng  Humidity Hotel Name
43         bechar      DZ  31.6167  -2.2167        31           
45         rawson      AR -43.3002 -65.1023        18           
65       tolanaro      MG -25.0319  46.9987        60           
88  sierra grande      AR -41.6060 -65.3557         9           
98      kalinagar      IN  28.6167  80.1000        57

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
#===>>>>>>> STARTED COPY-copy Pascaline
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories": categories,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

#end copy-copy

Starting hotel search
bechar - nearest hotel: Hôtel Madania
rawson - nearest hotel: Hotel Deportivo
tolanaro - nearest hotel: Hôtel Mahavokey
sierra grande - nearest hotel: Sierra Grande
kalinagar - nearest hotel: No hotel found
thohoyandou - nearest hotel: Khoroni Hotel Casino and Convention Resort
crane - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
sirte - nearest hotel: فندق المدينة-سرت
illizi - nearest hotel: بوناقة
adre - nearest hotel: No hotel found
nacimiento - nearest hotel: Residencial San Marcos
viedma - nearest hotel: Níjar
fada - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
tamanrasset - nearest hotel: فندق الأمان
ampanihy - nearest hotel: Hotel restaurant Angora
constitucion - nearest hotel: Hostal Alameda
mhamid - nearest hotel: Hotel Kasbah Azalay
minas de marcona - nearest hotel: No hotel found
letlhakeng - nearest hotel: No hotel found
riyadh - nearest hotel: White Palace Hotel
bam - nearest hote

City Country      Lat      Lng  Humidity       Hotel Name
43         bechar      DZ  31.6167  -2.2167        31    Hôtel Madania
45         rawson      AR -43.3002 -65.1023        18  Hotel Deportivo
65       tolanaro      MG -25.0319  46.9987        60  Hôtel Mahavokey
88  sierra grande      AR -41.6060 -65.3557         9    Sierra Grande
98      kalinagar      IN  28.6167  80.1000        57   No hotel found

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
#==>>> start copy-copy Pascaline

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size = "Humidity",
    scale = 1,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)